In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
from environs import Env
from adopt.malaria import window
from adopt.facebook.state import CampaignState

env = Env()
env.read_env('.env-template', override=True)
state = CampaignState(env, window(4))

In [27]:
import pandas as pd
from adopt.malaria import load_cities
from adopt.marketing import *

def get_control_cities():
    treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
    cities = load_cities('output/cities.csv')

    controls = cities \
        .merge(treatment_assignment) \
        .pipe(lambda df: df[df.treatment == 0]) \
        .pipe(lambda df: df.assign(rad = df.rad + 1))

    return controls

def lookup_region(state):
    regions = {
        'Uttar Pradesh': {
            "country": "IN",
            "key": "1754",
            "name": "Uttar Pradesh"
        },
        'Jharkhand': {
            "country": "IN",
            "key": "1756",
            "name": "Jharkhand"
        },
        'Chhatisgarh': {
            "country": "IN",
            "key": "1755",
            "name": "Chhattisgarh"
        }
    }

    regions = {**regions, 'combined': [v for v in regions.values()]}

    return regions[state]

def template_targeting(c: AdsetConf):
    targeting = {}

    excluded_locs = [create_location(lat, lng, rad)
                     for lat, lng, rad in c.excluded_locs]

    targeting[Targeting.Field.excluded_geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        TargetingGeoLocation.Field.custom_locations: excluded_locs
    }

    targeting[Targeting.Field.geo_locations] = {
        TargetingGeoLocation.Field.location_types: ['home'],
        TargetingGeoLocation.Field.regions: lookup_region(c.cluster.name)
    }

    return targeting

def make_template_adsets(env, state):
    controls = get_control_cities()

    # by_state = [(Cluster(st, st), [Location(r.lat, r.lng, r.rad) 
    #                                for _, r in df.iterrows()])
    #             for st, df in controls.groupby('state')]

    by_state = [(Cluster('combined', 'combined'), 
                 [Location(r.lat, r.lng, r.rad) 
                  for _, r in controls.iterrows()])]


    # quick hack - target and exclude locs are the same!
    confs = [AdsetConf(state.campaign, cl, None, lcs, [], 
                       10000, 'PAUSED', env('FACEBOOK_INSTA_ID'), 
                       48, None, None, None)
             for cl, lcs in by_state]

    confs = [(c, template_targeting(c)) for c in confs]

    instructions = [create_adset(f'ie-template-{c.cluster.name}', c, t) for c,t in confs]    

    return instructions

In [28]:
cities = load_cities('output/cities.csv')
treatment_assignment = pd.read_csv('outs/ma-with-treatment.csv')[['disthash', 'treatment']]
cities = cities.merge(treatment_assignment)

districts = cities.groupby('distname') \
                  .head(1) \
                  .reset_index(drop=True)

control_cities = get_control_cities()

In [ ]:
from adopt.facebook.update import GraphUpdater

instructions = make_template_adsets(env, state)

updater = GraphUpdater(state)

for i in instructions:
    report = updater.execute(i)
    print(report)

In [15]:
state.ads

In [ ]:
state.adsets